# smallest-free-number
A coding challenge to find the smallest non-negative integer that is not part of a given set of non-negative integers.

Thanks to [Nicolas Rinaudo](https://github.com/abcoates/smallest-free-number.git) for suggesting this problem for a coding challenge.

## Description
This is a simplification a general problem, find the 'least XXX' object that is not already used, where 'XXX' is some arbitrary measurement dimension such as 'large', 'expensive', etc.

In this minimal version, you are given a set (i.e. an **unordered** set) of non-negative integers, and you have to find the smallest non-negative integer that is **not** a part of the set.

## Examples
 * \[0, 1, 2, 3, 5\] => 4
 * \[0, 1, 3, 4, 5\] => 2
 * \[2, 1, 0\] => 3
 * \[20, 10, 30\] => 0

## Special Note
You **may** use AI to help you write the code.  As AI coding companions are inevitable, we might as well all start practicising how to use them.  **However**, if you have used AI to help you write your code, please make that clear in your solution.

## Stretch Goal
Finding a solution is straightforward, but can you find a solution that only takes linear time?  Measure how the time taken for your solution varies as the size of the set is increased, and see how close you can get to the time taken growing linearly with the size of the set.  Create a graph of the time measurements again set size.

You will need to generate unordered sets of numbers that are sufficiently large to make the solution time sufficiently measurable.

Your results may vary with the **density** of the sets, i.e. with the percentage of unused numbers in the range from zero to the largest number in the set.

## Solution - F#

### Solution #1 - Brute Force

As a first solution, let's try something "brute force" - sort the set, then walk up the integers from zero until we find one that isn't in the set.

In [1]:
let solution1 (intset: int list): int = // 'intset' must be a set in unordered list format - to avoid pre-sorted F# sets giving an advantage
    let sortedset = intset |> List.sort
    let rec find (sortedlist: int list) (nextValue: int) =
        match sortedlist with
        | [] -> nextValue
        | other ->
            let listhead = sortedlist |> List.head
            if (nextValue < listhead)
            then nextValue
            elif (nextValue = listhead)
            then find (sortedlist |> List.tail) (nextValue+1)
            else find(sortedlist |> List.tail) nextValue
    find sortedset 0

\[0, 1, 2, 3, 5\] => 4

In [2]:
solution1 [0;1;2;3;5]

4

\[0, 1, 3, 4, 5\] => 2

In [3]:
solution1 [0;1;3;4;5]

2

\[2, 1, 0\] => 3

In [4]:
solution1 [2;1;0]

3

\[20, 10, 30\] => 0

In [5]:
solution1 [20;10;30]

0

So 'solution1' works correctly.  However, how performant is it?

We'll need a graphing package - 'XPlot' will do nicely.

In [6]:
#r "nuget: XPlot.Plotly"
#r "nuget: XPlot.Plotly.Interactive"

open XPlot.Plotly

Installed Packages XPlot.Plotly, 4.0.6 XPlot.Plotly.Interactive, 4.0.7

Loading extensions from `C:\Users\tony\.nuget\packages\xplot.plotly.interactive\4.0.7\lib\net7.0\XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

Let's also get a package for curve fitting.

In [7]:
#r "nuget: MathNet.Numerics"

open MathNet.Numerics

Installed Packages MathNet.Numerics, 5.0.0

We'll need functions that can generate random sets of non-negative integers.

**However**, F# seems to implement sets as sorted lists, which makes it too easy to find the smallest unused integer.  As such, I will need to return the set as a list in which the order of the elements has been randomised.

**Update:** instead, I'll just generate the worst-case, highest-to-lowest set of integers.

In [8]:
// open System

// let randomGenerator = Random()

In [9]:
// let generateSet (maximum:int) (length:int): int list =
//     let rec iterate (currentList: int list) (currentSet: int Set): int list =
//         match currentSet with
//         | finished when (finished |> Set.count) >= length -> currentList
//         | unfinished ->
//             let newVal = randomGenerator.Next(0, maximum+1)
//             if (currentSet |> Set.contains newVal)
//             then
//                 iterate currentList currentSet
//             else
//                 iterate (newVal::currentList) (currentSet |> Set.add newVal)
//     iterate [] ([] |> Set.ofList)

In [10]:
let generateSet (length:int): int list = seq {(length-1)..(-1)..0} |> Seq.toList

In [11]:
let set1 = generateSet 5
set1

[ 4, 3, 2, 1, 0 ] HeadOrDefault 4 TailOrNull [ 3, 2, 1, 0 ] HeadOrDefault 3 TailOrNull [ 2, 1, 0 ] HeadOrDefault 2 TailOrNull [ 1, 0 ] HeadOrDefault 1 TailOrNull [ 0 ] HeadOrDefault 0 TailOrNull [ ] HeadOrDefault 0 TailOrNull <null> Head System.InvalidOperationException: The input list was empty.
 at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141
 at lambda_method105(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 Tail System.InvalidOperationException: The input list was empty.
 at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4146
 at lambda_method106(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 (values) [ ] Head 0 Tail [ ] HeadOrDefault 0 TailOrNull <null> Head System.InvalidOperationException: The input list was empty.
 at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141
 at lambda_method105(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 Tail System.InvalidOperationException: The input list was empty.
 at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4146
 at lambda_method106(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 (values) [ ] (values) [ 0 ] Head 1 Tail [ 0 ] HeadOrDefault 0 TailOrNull [ ] HeadOrDefault 0 TailOrNull <null> Head System.InvalidOperationException: The input list was empty.
 at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141
 at lambda_method105(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 Tail System.InvalidOperationException: The input list was empty.
 at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4146
 at lambda_method106(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 (values) [ ] Head 0 Tail [ ] HeadOrDefault 0 TailOrNull <null> Head System.InvalidOperationException: The input list was empty.
 at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141
 at lambda_method105(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 Tail System.InvalidOperationException: The input list was empty.
 at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4146
 at lambda_method106(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 (values) [ ] (values) [ 0 ] (values) [ 1, 0 ] Head 2 Tail [ 1, 0 ] HeadOrDefault 1 TailOrNull [ 0 ] HeadOrDefault 0 TailOrNull [ ] HeadOrDefault 0 TailOrNull <null> Head System.InvalidOperationException: The input list was empty.
 at Microso

In [12]:
solution1 set1

5

We will do many runs where the timing is an integer number of microseconds, so we will need a quick way to calculate the integer average time of those runs.

**Update:** no, we'll only do a single worst-case run.

### Time Solution #1

The following function can be used to time how long a solution takes to run.  It averages the time over ten runs.

In [13]:
let timeSolution (length: int) (solution: int list -> int): int*int =
    let timeOnce (): int*int*int =
        GC.Collect()
        let testSet = generateSet length
        let sw = System.Diagnostics.Stopwatch.StartNew()
        let result = solution testSet
        sw.Stop()
        (length, result, sw.Elapsed.Microseconds)
    for counter in 1..20 do // warm-up
        timeOnce () |> ignore
    let timings = seq {
        for counter in 1..10 do
            let (_, _, timing) = timeOnce ()
            yield timing
    }
    (length, ((timings |> Seq.sum)+5)/10)

In [14]:
timeSolution 100 solution1

Item1,100
Item2,18


In [15]:
timeSolution 100 solution1

Item1,100
Item2,16


In [16]:
timeSolution 200 solution1

Item1,200
Item2,26


In [17]:
timeSolution 200 solution1

Item1,200
Item2,20


Let's create timings (in microseconds) for set sizes up to 5 000.  To make it as hard as possible, we'll set 'maximum = length', meaning that there is only one number in the range '0..maximum' which does not appear in the set.

In [18]:
let maxLength = 30000
let graphSegments = 600
let increment = maxLength / graphSegments
let lengths = {increment..increment..maxLength} |> Seq.toList
let maximums = lengths
let pairs: (int*int) list = List.zip maximums lengths
let timings = pairs |> List.map (fun pair ->
    let (maximum, length) = pair
    printfn "Timing length %d ..." length
    let (_, timing) = timeSolution length solution1
    timing
)
printfn "Done."

Timing length 50 ...
Timing length 100 ...
Timing length 150 ...
Timing length 200 ...
Timing length 250 ...
Timing length 300 ...
Timing length 350 ...
Timing length 400 ...
Timing length 450 ...
Timing length 500 ...
Timing length 550 ...
Timing length 600 ...
Timing length 650 ...
Timing length 700 ...
Timing length 750 ...
Timing length 800 ...
Timing length 850 ...
Timing length 900 ...
Timing length 950 ...
Timing length 1000 ...
Timing length 1050 ...
Timing length 1100 ...
Timing length 1150 ...
Timing length 1200 ...
Timing length 1250 ...
Timing length 1300 ...
Timing length 1350 ...
Timing length 1400 ...
Timing length 1450 ...
Timing length 1500 ...
Timing length 1550 ...
Timing length 1600 ...
Timing length 1650 ...
Timing length 1700 ...
Timing length 1750 ...
Timing length 1800 ...
Timing length 1850 ...
Timing length 1900 ...
Timing length 1950 ...
Timing length 2000 ...
Timing length 2050 ...
Timing length 2100 ...
Timing length 2150 ...
Timing length 2200 ...
Timing l

In [19]:
List.zip lengths timings

[ (50, 11), (100, 14), (150, 20), (200, 21), (250, 23), (300, 20), (350, 27), (400, 28), (450, 30), (500, 26), (550, 29), (600, 40), (650, 37), (700, 37), (750, 39), (800, 36), (850, 36), (900, 40), (950, 41), (1000, 53) ... (more) ] HeadOrDefault (50, 11) Item1 50 Item2 11 TailOrNull [ (100, 14), (150, 20), (200, 21), (250, 23), (300, 20), (350, 27), (400, 28), (450, 30), (500, 26), (550, 29), (600, 40), (650, 37), (700, 37), (750, 39), (800, 36), (850, 36), (900, 40), (950, 41), (1000, 53), (1050, 51) ... (more) ] HeadOrDefault (100, 14) Item1 100 Item2 14 TailOrNull [ (150, 20), (200, 21), (250, 23), (300, 20), (350, 27), (400, 28), (450, 30), (500, 26), (550, 29), (600, 40), (650, 37), (700, 37), (750, 39), (800, 36), (850, 36), (900, 40), (950, 41), (1000, 53), (1050, 51), (1100, 63) ... (more) ] HeadOrDefault (150, 20) Item1 150 Item2 20 TailOrNull [ (200, 21), (250, 23), (300, 20), (350, 27), (400, 28), (450, 30), (500, 26), (550, 29), (600, 40), (650, 37), (700, 37), (750, 39), (800, 36), (850, 36), (900, 40), (950, 41), (1000, 53), (1050, 51), (1100, 63), (1150, 53) ... (more) ] HeadOrDefault (200, 21) Item1 200 Item2 21 TailOrNull [ (250, 23), (300, 20), (350, 27), (400, 28), (450, 30), (500, 26), (550, 29), (600, 40), (650, 37), (700, 37), (750, 39), (800, 36), (850, 36), (900, 40), (950, 41), (1000, 53), (1050, 51), (1100, 63), (1150, 53), (1200, 51) ... (more) ] HeadOrDefault (250, 23) Item1 250 Item2 23 TailOrNull [ (300, 20), (350, 27), (400, 28), (450, 30), (500, 26), (550, 29), (600, 40), (650, 37), (700, 37), (750, 39), (800, 36), (850, 36), (900, 40), (950, 41), (1000, 53), (1050, 51), (1100, 63), (1150, 53), (1200, 51), (1250, 55) ... (more) ] HeadOrDefault (300, 20) TailOrNull [ (350, 27), (400, 28), (450, 30), (500, 26), (550, 29), (600, 40), (650, 37), (700, 37), (750, 39), (800, 36), (850, 36), (900, 40), (950, 41), (1000, 53), (1050, 51), (1100, 63), (1150, 53), (1200, 51), (1250, 55), (1300, 53) ... (more) ] Head (300, 20) Tail [ (350, 27), (400, 28), (450, 30), (500, 26), (550, 29), (600, 40), (650, 37), (700, 37), (750, 39), (800, 36), (850, 36), (900, 40), (950, 41), (1000, 53), (1050, 51), (1100, 63), (1150, 53), (1200, 51), (1250, 55), (1300, 53) ... (more) ] (values) index value 0 (300, 20) 1 (350, 27) 2 (400, 28) 3 (450, 30) 4 (500, 26) 5 (550, 29) 6 (600, 40) 7 (650, 37) 8 (700, 37) 9 (750, 39) 10 (800, 36) 11 (850, 36) 12 (900, 40) 13 (950, 41) 14 (1000, 53) 15 (1050, 51) 16 (1100, 63) 17 (1150, 53) 18 (1200, 51) 19 (1250, 55) ... (more) Head (250, 23) Item1 250 Item2 23 Tail [ (300, 20), (350, 27), (400, 28), (450, 30), (500, 26), (550, 29), (600, 40), (650, 37), (700, 37), (750, 39), (800, 36), (850, 36), (900, 40), (950, 41), (1000, 53), (1050, 51), (1100, 63), (1150, 53), (1200, 51), (1250, 55) ... (more) ] HeadOrDefault (300, 20) TailOrNull [ (350, 27), (400, 28), (450, 30), (500, 26), (550, 29), (600, 40), (650, 37), (700, 37), (750, 39), (800, 36), (850, 36), (900, 40), (950, 41), (1000, 53), (1050, 51), (1100, 63), (1150, 53), (1200, 51), (1250, 55), (1300, 53) ... (more) ] Head (300, 20) Tail [ (350, 27), (400, 28), (450, 30), (500, 26), (550, 29), (600, 40), (650, 37), (700, 37), (750, 39), (800, 36), (850, 36), (900, 40), (950, 41), (1000, 53), (1050, 51), (1100, 63), (1150, 53), (1200, 51), (1250, 55), (1300, 53) ... (more) ] (values) index value 0 (300, 20) 1 (350, 27) 2 (400, 28) 3 (450, 30) 4 (500, 26) 5 (550, 29) 6 (600, 40) 7 (650, 37) 8 (700, 37) 9 (750, 39) 10 (800, 36) 11 (850, 36) 12 (900, 40) 13 (950, 41) 14 (1000, 53) 15 (1050, 51) 16 (1100, 63) 17 (1150, 53) 18 (1200, 51) 19 (1250, 55) ... (more) (values) index value 0 (250, 23) Item1 250 Item2 23 1 (300, 20) Item1 300 Item2 20 2 (350, 27) Item1 350 Item2 27 3 (400, 28) Item1 400 Item2 28 4 (450, 30) Item1 450 Item2 30 5 (500, 26) Item1 500 Item2 26 6 (550, 29) Item1 550 Item2 29 7 (600, 40) Item1 600 Item2 40 8 (650, 37) Item1 650 Item2 37 9 (700, 37) Item1 700 Item2 37 10 (750, 39) Item1 750 Item2 39 11 (800, 36) Ite

Let's see what the total elapsed calculation time in seconds is.

In [20]:
let totalElapsedCalculationTimeInMicrosec = (timings |> List.sum) * 20 |> float
let totalElapsedCalculationTimeInSeconds = totalElapsedCalculationTimeInMicrosec / 1000000.
totalElapsedCalculationTimeInSeconds

4.59182

Now let's try fitting an O(n^q) curve to the results.  I use a semi-calculated, semi-empirical starting point, with a starting order of O(n^1.5).

In [21]:
let lengthsFloat = lengths |> List.map float |> Array.ofList
let timingsFloat = timings |> List.map float |> Array.ofList

let first<'a> (arr: 'a[]) = arr[0]
let last<'a> (arr: 'a[]) = arr[arr.Length - 1]

// These scaling factors are used to give the parameters p0, p1 and p2 similar magnitudes, which tends to work best with fitting algorithms.
let p2scale = 0.75 // assuming O(1.5) as a starting point
let p1scale = ((last timingsFloat) - (first timingsFloat)) / ((last lengthsFloat) - (first lengthsFloat))
let p0scale = (first timingsFloat) - p1scale * (first lengthsFloat)
(p0scale, p1scale, p2scale)

Item1,9.864774624373958
Item2,0.02270450751252087
Item3,0.75


In [22]:

let fitFunc (p0:float) (p1:float) (p2:float) (x:float) =
    // printfn "p0 = %f, p1 = %f, p2 = %f, x = %f" p0 p1 p2 x
    (p0*p0scale) + (p1*p1scale) * x**(p2*p2scale)

let p0init = 1.
let p1init = 1.
let p2init = 2. // assuming O(1.5) as a starting point
let tolerance = 0.1
let maxIterations = 10000
(p0init, p1init, p2init)

Item1,1
Item2,1
Item3,2


In [23]:
let (p0, p1, p2) = Fit.Curve(lengthsFloat, timingsFloat, fitFunc, p0init, p1init, p2init, tolerance, maxIterations).ToTuple()
(p0, p1, p2)

Item1,1.1333333333333322
Item2,1.0666666666666664
Item3,1.3333333333333317


OK, let's plot the measurements and the fit together.

In [24]:
let measurementCurve = Scatter(x = lengths, y = timings, name="Measurements")

let fitLengths = {(lengths |> List.min)..(lengths |> List.max)} |> Seq.map float |> Array.ofSeq
let fitTimings = fitLengths |> Array.map (fun x -> fitFunc p0 p1 p2 x)
let fitCurve = Scatter(x = fitLengths, y = fitTimings, name="O(n) Fit Curve")

[measurementCurve; fitCurve]
|> Chart.Plot
|> Chart.WithTitle("Solution Time vs. Set Size")
|> Chart.WithXTitle("Set Size")
|> Chart.WithYTitle("Solution Time in Microsec")

<!DOCTYPE html>

In [25]:
let q = p2*p2scale
q

0.9999999999999988

So this is O(n), so a high degree of precision.

## Solution #2 - More Functional - no use of unique sets

In [26]:
let solution2 (intset: int list): int =
    let rec find (intset: int list) (next: int): int =
        match intset with
        | [] -> next
        | head::tail when head = next -> find tail (next+1)
        | other ->
            let remainderLow, remainderHigh = intset |> List.partition (fun n -> n <= next)
            if (remainderLow |> List.isEmpty)
            then next
            else find remainderHigh (next+1)
    find intset 0

\[0, 1, 2, 3, 5\] => 4

In [27]:
solution2 [0;1;2;3;5]

4

\[0, 1, 3, 4, 5\] => 2

In [28]:
solution2 [0;1;3;4;5]

2

\[2, 1, 0\] => 3

In [29]:
solution2 [2;1;0]

3

\[20, 10, 30\] => 0

In [30]:
solution2 [20;10;30]

0

### Time Solution #2

In [31]:
timeSolution 100 solution2

Item1,100
Item2,137


In [32]:
timeSolution 100 solution1

Item1,100
Item2,5


In [33]:
timeSolution 200 solution1

Item1,200
Item2,9


In [34]:
timeSolution 200 solution1

Item1,200
Item2,10


In [35]:
let maxLength = 15000
let graphSegments = 300
let increment = maxLength / graphSegments
let lengths = {increment..increment..maxLength} |> Seq.toList
let maximums = lengths
let pairs: (int*int) list = List.zip maximums lengths
let timings = pairs |> List.map (fun pair ->
    let (maximum, length) = pair
    printfn "Timing length %d ..." length
    let (_, timing) = timeSolution length solution2
    timing
)
printfn "Done."

Timing length 50 ...
Timing length 100 ...
Timing length 150 ...
Timing length 200 ...
Timing length 250 ...
Timing length 300 ...
Timing length 350 ...
Timing length 400 ...
Timing length 450 ...
Timing length 500 ...
Timing length 550 ...
Timing length 600 ...
Timing length 650 ...
Timing length 700 ...
Timing length 750 ...
Timing length 800 ...
Timing length 850 ...
Timing length 900 ...
Timing length 950 ...
Timing length 1000 ...
Timing length 1050 ...
Timing length 1100 ...
Timing length 1150 ...
Timing length 1200 ...
Timing length 1250 ...
Timing length 1300 ...
Timing length 1350 ...
Timing length 1400 ...
Timing length 1450 ...
Timing length 1500 ...
Timing length 1550 ...
Timing length 1600 ...
Timing length 1650 ...
Timing length 1700 ...
Timing length 1750 ...
Timing length 1800 ...
Timing length 1850 ...
Timing length 1900 ...
Timing length 1950 ...
Timing length 2000 ...
Timing length 2050 ...
Timing length 2100 ...
Timing length 2150 ...
Timing length 2200 ...
Timing l

In [43]:
List.zip lengths timings

[ (50, 44), (100, 137), (150, 291), (200, 599), (250, 760), (300, 609), (350, 294), (400, 714), (450, 321), (500, 684), (550, 441), (600, 515), (650, 490), (700, 348), (750, 388), (800, 768), (850, 722), (900, 273), (950, 461), (1000, 593) ... (more) ] HeadOrDefault (50, 44) Item1 50 Item2 44 TailOrNull [ (100, 137), (150, 291), (200, 599), (250, 760), (300, 609), (350, 294), (400, 714), (450, 321), (500, 684), (550, 441), (600, 515), (650, 490), (700, 348), (750, 388), (800, 768), (850, 722), (900, 273), (950, 461), (1000, 593), (1050, 411) ... (more) ] HeadOrDefault (100, 137) Item1 100 Item2 137 TailOrNull [ (150, 291), (200, 599), (250, 760), (300, 609), (350, 294), (400, 714), (450, 321), (500, 684), (550, 441), (600, 515), (650, 490), (700, 348), (750, 388), (800, 768), (850, 722), (900, 273), (950, 461), (1000, 593), (1050, 411), (1100, 519) ... (more) ] HeadOrDefault (150, 291) Item1 150 Item2 291 TailOrNull [ (200, 599), (250, 760), (300, 609), (350, 294), (400, 714), (450, 321), (500, 684), (550, 441), (600, 515), (650, 490), (700, 348), (750, 388), (800, 768), (850, 722), (900, 273), (950, 461), (1000, 593), (1050, 411), (1100, 519), (1150, 461) ... (more) ] HeadOrDefault (200, 599) Item1 200 Item2 599 TailOrNull [ (250, 760), (300, 609), (350, 294), (400, 714), (450, 321), (500, 684), (550, 441), (600, 515), (650, 490), (700, 348), (750, 388), (800, 768), (850, 722), (900, 273), (950, 461), (1000, 593), (1050, 411), (1100, 519), (1150, 461), (1200, 549) ... (more) ] HeadOrDefault (250, 760) Item1 250 Item2 760 TailOrNull [ (300, 609), (350, 294), (400, 714), (450, 321), (500, 684), (550, 441), (600, 515), (650, 490), (700, 348), (750, 388), (800, 768), (850, 722), (900, 273), (950, 461), (1000, 593), (1050, 411), (1100, 519), (1150, 461), (1200, 549), (1250, 373) ... (more) ] HeadOrDefault (300, 609) TailOrNull [ (350, 294), (400, 714), (450, 321), (500, 684), (550, 441), (600, 515), (650, 490), (700, 348), (750, 388), (800, 768), (850, 722), (900, 273), (950, 461), (1000, 593), (1050, 411), (1100, 519), (1150, 461), (1200, 549), (1250, 373), (1300, 721) ... (more) ] Head (300, 609) Tail [ (350, 294), (400, 714), (450, 321), (500, 684), (550, 441), (600, 515), (650, 490), (700, 348), (750, 388), (800, 768), (850, 722), (900, 273), (950, 461), (1000, 593), (1050, 411), (1100, 519), (1150, 461), (1200, 549), (1250, 373), (1300, 721) ... (more) ] (values) index value 0 (300, 609) 1 (350, 294) 2 (400, 714) 3 (450, 321) 4 (500, 684) 5 (550, 441) 6 (600, 515) 7 (650, 490) 8 (700, 348) 9 (750, 388) 10 (800, 768) 11 (850, 722) 12 (900, 273) 13 (950, 461) 14 (1000, 593) 15 (1050, 411) 16 (1100, 519) 17 (1150, 461) 18 (1200, 549) 19 (1250, 373) ... (more) Head (250, 760) Item1 250 Item2 760 Tail [ (300, 609), (350, 294), (400, 714), (450, 321), (500, 684), (550, 441), (600, 515), (650, 490), (700, 348), (750, 388), (800, 768), (850, 722), (900, 273), (950, 461), (1000, 593), (1050, 411), (1100, 519), (1150, 461), (1200, 549), (1250, 373) ... (more) ] HeadOrDefault (300, 609) TailOrNull [ (350, 294), (400, 714), (450, 321), (500, 684), (550, 441), (600, 515), (650, 490), (700, 348), (750, 388), (800, 768), (850, 722), (900, 273), (950, 461), (1000, 593), (1050, 411), (1100, 519), (1150, 461), (1200, 549), (1250, 373), (1300, 721) ... (more) ] Head (300, 609) Tail [ (350, 294), (400, 714), (450, 321), (500, 684), (550, 441), (600, 515), (650, 490), (700, 348), (750, 388), (800, 768), (850, 722), (900, 273), (950, 461), (1000, 593), (1050, 411), (1100, 519), (1150, 461), (1200, 549), (1250, 373), (1300, 721) ... (more) ] (values) index value 0 (300, 609) 1 (350, 294) 2 (400, 714) 3 (450, 321) 4 (500, 684) 5 (550, 441) 6 (600, 515) 7 (650, 490) 8 (700, 348) 9 (750, 388) 10 (800, 768) 11 (850, 722) 12 (900, 273) 13 (950, 461) 14 (1000, 593) 15 (1050, 411) 16 (1100, 519) 17 (1150, 461) 18 (1200, 549) 19 (1250, 373) ... (more) (values) index value 0 (250, 760) Item1 250 Item2 760 1 (300, 609) Item1 300 Item2 609 2 (350, 294) Item1 

Let's see what the total elapsed calculation time in seconds is.

In [44]:
let totalElapsedCalculationTimeInMicrosec = (timings |> List.sum) * 20 |> float
let totalElapsedCalculationTimeInSeconds = totalElapsedCalculationTimeInMicrosec / 1000000.
totalElapsedCalculationTimeInSeconds

3.00734

Now let's try fitting an O(n^q) curve to the results.  I use a semi-calculated, semi-empirical starting point, with a starting order of O(n^1.5).

In [45]:
let lengthsFloat = lengths |> List.map float |> Array.ofList
let timingsFloat = timings |> List.map float |> Array.ofList

let first<'a> (arr: 'a[]) = arr[0]
let last<'a> (arr: 'a[]) = arr[arr.Length - 1]

// These scaling factors are used to give the parameters p0, p1 and p2 similar magnitudes, which tends to work best with fitting algorithms.
let p2scale = 0.75 // assuming O(1.5) as a starting point
let p1scale = ((last timingsFloat) - (first timingsFloat)) / ((last lengthsFloat) - (first lengthsFloat))
let p0scale = (first timingsFloat) - p1scale * (first lengthsFloat)
(p0scale, p1scale, p2scale)

Item1,42.5685618729097
Item2,0.02862876254180602
Item3,0.75


In [46]:

let fitFunc (p0:float) (p1:float) (p2:float) (x:float) =
    // printfn "p0 = %f, p1 = %f, p2 = %f, x = %f" p0 p1 p2 x
    (p0*p0scale) + (p1*p1scale) * x**(p2*p2scale)

let p0init = 1.
let p1init = 1.
let p2init = 2. // assuming O(1.5) as a starting point
let tolerance = 0.1
let maxIterations = 10000
(p0init, p1init, p2init)

Item1,1
Item2,1
Item3,2


In [47]:
let (p0, p1, p2) = Fit.Curve(lengthsFloat, timingsFloat, fitFunc, p0init, p1init, p2init, tolerance, maxIterations).ToTuple()
(p0, p1, p2)

Item1,1.0766975308641964
Item2,1.0762345679012342
Item3,1.3913580246913564


OK, let's plot the measurements and the fit together.

In [48]:
let measurementCurve = Scatter(x = lengths, y = timings, name="Measurements")

let fitLengths = {(lengths |> List.min)..(lengths |> List.max)} |> Seq.map float |> Array.ofSeq
let fitTimings = fitLengths |> Array.map (fun x -> fitFunc p0 p1 p2 x)
let fitCurve = Scatter(x = fitLengths, y = fitTimings, name="O(n) Fit Curve")

[measurementCurve; fitCurve]
|> Chart.Plot
|> Chart.WithTitle("Solution Time vs. Set Size")
|> Chart.WithXTitle("Set Size")
|> Chart.WithYTitle("Solution Time in Microsec")

<!DOCTYPE html>

In [49]:
let q = p2*p2scale
q

1.0435185185185172

So weirdly, and O(n) curve doesn't seem to fit at all, it seems like it's O(1) (constant time) above a low limit.  I don't understand how that can really happen.

Oddly, I would have expected this method to be slower than the first, due to *not* using sets, but it actually seems faster at higher lengths!

## Appendix

**Note from Nicolas** If your set length is N, then your solution value must be in the range 0..N.